In [ ]:
!pip install facenet-pytorch

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import copy
import tqdm
import torchvision
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from facenet_pytorch import InceptionResnetV1
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.getcwd())
for dirname, dirs, _ in os.walk('../input'):
    for d in dirs:
        print(os.path.join(dirname, d))

I try to train the fake classifier using transform learning

First lets define my dataset and data loader

In [ ]:
# Transform the image in the way facenet_pytorch does
class ImgTransform:
    def __call__(self, x):
        np_img = np.array(x).astype(np.float32)
        np_img = (np_img - 127.5) / 128.0
        np_img = np.transpose(np_img, axes=[2, 0, 1])
        return np_img


# ImageFolder returns 0 in case of fake and 1 in case of real. We want the opposite
class LabelTransform:
    def __call__(self, x):
        return np.array((x+1) & 1, dtype=np.float32)


img_transfrom = ImgTransform()
label_transform = LabelTransform()

train_dataset = torchvision.datasets.ImageFolder('../input/dfdc-faces-of-the-train-sample/train', transform=img_transfrom, target_transform=label_transform)
valid_dataset = torchvision.datasets.ImageFolder('../input/dfdc-faces-of-the-train-sample/validation', transform=img_transfrom, target_transform=label_transform)

# Lets look at an example of image loaded
img, lbl = train_dataset[0]
img = np.transpose(img, axes=[1, 2, 0])
pil_img = Image.fromarray(((img+1)*128).astype(np.uint8))
display(pil_img)

train_loader = torch.utils.data.DataLoader(train_dataset, 32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, 32, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Lets load the model

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=1).eval()

The following code was taken from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html. I changed it a little for my needs.

In [ ]:
def train_model(model, train_criterion, valid_criterion, optimizer, scheduler=None, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000.0

    dataset_sizes = {'train': len(train_dataset), 'val': len(valid_dataset)}
    dataloaders = {'train': train_loader, 'val': valid_loader}
    loos_func = {'train': train_criterion, 'val': valid_criterion}
    epoch_losses = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            # for inputs, labels in tqdm.tqdm(dataloaders[phase]):
            for inputs, labels in dataloaders[phase]:
                labels = labels.reshape(len(labels), 1)
                torch.cuda.empty_cache()
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = loos_func[phase](outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                n_real = (labels == 0).sum()
                n_fake = (labels == 1).sum()
                avg_factor = n_real+n_fake*loos_func[phase].pos_weight
                avg_loss = loss / avg_factor
                running_loss += avg_loss.item() * inputs.size(0)
                if phase == 'train' and scheduler is not None:
                    scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_losses[phase].append(epoch_loss)

            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'resnet_params.pth')

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_losses['train'],  epoch_losses['val']

In [ ]:
# Plot the history of the loss values in training epoches
def plot_history(train_history, val_history):
    epoches = [i for i in range(len(train_history))]
    plt.plot(epoches, train_history, label='train')
    plt.plot(epoches, val_history, label='validation')
    plt.legend()
    plt.xlabel('epoch numbers')
    plt.ylabel('loss value')
    plt.xticks(epoches)
    plt.show()


According to facenet_pytorch help, the last layer of resnet has random weights.
Firsts, lets train the model so that only the last layer change its weight to refine them to something reasonable

In [ ]:
resnet = resnet.to(device)
resnet_layers = [module for module in resnet.modules()]
for param in resnet.parameters():
    param.requires_grad = False
active_params = list(resnet_layers[-1].parameters())
for param in active_params:
    param.requires_grad = True

In [ ]:
train_w = torch.Tensor([20699/73154]).to(device) # There are 73154 fake examples and only 20699 in the training set
val_w = torch.Tensor([6029/24765]).to(device) # There are 24765 fake examples and only 6029 in the validation set
train_criterion = nn.BCEWithLogitsLoss(pos_weight=train_w, reduction='sum')
valid_criterion = nn.BCEWithLogitsLoss(pos_weight=val_w, reduction='sum')
optimizer = optimizer_conv = optim.SGD(active_params, lr=0.001, momentum=0.9)
resnet, train_history, val_history = train_model(resnet, train_criterion, valid_criterion, optimizer, num_epochs=5)


In [ ]:
# Let's visualize the traiaing history
plot_history(train_history, val_history)

Now add more layers to the training and save the last layer

In [ ]:
active_params = []
for layer in resnet_layers[-3:]:
    active_params += layer.parameters()
for param in active_params:
    param.requires_grad = True
optimizer = optimizer_conv = optim.SGD(active_params, lr=0.000025, momentum=0.9)
resnet, train_history, test_history = train_model(resnet, train_criterion, valid_criterion, optimizer, num_epochs=20)

In [ ]:
# Let's visualize the traiaing history
plot_history(train_history, val_history)